In [1]:
%matplotlib widget

In [2]:
import os
from glob import glob

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from astropy.io import fits
from astropy.wcs import wcs
import ipywidgets as widgets
from IPython.display import display, Image, clear_output

from vasca.resource_manager import ResourceManager
import vasca.utils as vutils

In [3]:
# Settings

# Input/output directories
with ResourceManager() as rm:
    root_data_dir = rm.get_path("gal_ds_fields", "lustre")
    
#root_data_dir = "/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields"

# Dry-run, don't export final list
dry_run = False

# Debugging switch
is_debug = False

hide_progress = False

refresh = False

In [4]:
coadd_cnt_paths = list()
coadd_rrhr_paths = list()

# Loops over drift scan directories
scan_names = [path.split(os.sep)[-1] for path in glob(f"{root_data_dir}/*")]
for idx_scan, scan_name in tqdm(enumerate(scan_names), total=len(scan_names), desc="Scans", disable=hide_progress):
    # Debugging
    if idx_scan > 0 and is_debug: break
    
    # Loops over fields
    field_names = [path.split(os.sep)[-1] for path in glob(f"{root_data_dir}/{scan_name}/*")]
    for idx_field, field_name in tqdm(enumerate(field_names), total=len(field_names), desc="Fields", disable=hide_progress):
        # Debugging
        if idx_field > 0 and is_debug: break
        
        # Collects file paths accross all visit directories
        # Counts maps
        img_int_paths = glob(f"{root_data_dir}/{scan_name}/{field_name}/*/*-nd-cnt.fits.gz")
        # Effecitve exposure time map
        img_rrhr_paths = glob(f"{root_data_dir}/{scan_name}/{field_name}/*/*-nd-rrhr.fits.gz")

        # Stack image maps
        # Loops over image types
        for path_list, path_out_file_suffix, coadd_path_list in zip(
            [img_int_paths, img_rrhr_paths],
            ["-nd-cnt-coadd.fits", "-nd-rrhr-coadd.fits"],
            [coadd_cnt_paths, coadd_rrhr_paths],
        ):
            path_out_file = f"{root_data_dir}/{scan_name}/{field_name}/{field_name}{path_out_file_suffix}"
            if not refresh and not os.path.isfile(path_out_file):
                # Loops over individual files
                for idx_img, path in enumerate(path_list):

                    # Initialize stack image from first file
                    with fits.open(path) as hdul:
                        if idx_img == 0:
                            img = hdul[0].data
                            img_wcs = wcs.WCS(hdul[0].header)
                        else:
                            img += hdul[0].data

                # Export

                #img_hdr = fits.Header()
                #img_hdr = default_hdr
                hdu = fits.CompImageHDU(img, header=img_wcs.to_header())
                hdu.writeto(path_out_file, overwrite=True)
            
            coadd_path_list.append(path_out_file)


Scans:   0%|          | 0/2 [00:00<?, ?it/s]

Fields:   0%|          | 0/14 [00:00<?, ?it/s]

Fields:   0%|          | 0/9 [00:00<?, ?it/s]

In [5]:
coadd_cnt_paths

['/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29208-KEPLER_SCAN_009_sv08/29208-KEPLER_SCAN_009_sv08-nd-cnt-coadd.fits',
 '/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29208-KEPLER_SCAN_009_sv06/29208-KEPLER_SCAN_009_sv06-nd-cnt-coadd.fits',
 '/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29208-KEPLER_SCAN_009_sv01/29208-KEPLER_SCAN_009_sv01-nd-cnt-coadd.fits',
 '/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29208-KEPLER_SCAN_009_sv07/29208-KEPLER_SCAN_009_sv07-nd-cnt-coadd.fits',
 '/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29208-KEPLER_SCAN_009_sv09/29208-KEPLER_SCAN_009_sv09-nd-cnt-coadd.fits',
 '/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29208-KEPLER_SCAN_009_sv14/29208-KEPLER_SCAN_009_sv14-nd-cnt-coadd.fits',
 '/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29208-KEPL

In [6]:
def display_fits_img(path):
    
    file_name = path.split(os.sep)[-1]
    if file_name.endswith("gz"):
        file_name.rstrip(".fits.gz")
    else:
        file_name.rstrip(".fits")
        
    plot_name = file_name
    
    with fits.open(path) as hdul:
        img = hdul[1].data
        img_wcs = wcs.WCS(hdul[1].header)
    
    fig, ax = vutils.nb_fig(
        num=plot_name,
        figsize=(8,8),
        layout="tight",
        subplot_kw={'projection': img_wcs}
    )

    # image
    ax.imshow(img, norm=LogNorm(), cmap="gray_r")
    
    # modify grid
    ax.coords.grid(True, color='gray', lw=0.75, ls='-', zorder=0)

    # set axis labels
    ra = ax.coords["ra"]
    dec = ax.coords["dec"]
    ra.set_major_formatter("d.dd")
    dec.set_major_formatter("d.dd")

    label_fontsize = 10
    ax.set_xlabel("Ra", fontsize=label_fontsize)
    ax.set_ylabel("Dec", fontsize=label_fontsize)
    ax.xaxis.set_tick_params(labelsize=label_fontsize)
    ax.yaxis.set_tick_params(labelsize=label_fontsize)
    ax.tick_params(axis="x", labelsize=label_fontsize, direction="in", bottom=True, top=False)
    ax.tick_params(axis="y", labelsize=label_fontsize, direction="in", left=True, right=False)
    plt.show()


In [7]:
# Display image function
def display_image(image_path):
    with image_output:
        clear_output(wait=True)
        plt.close(plt.gcf())
        display_fits_img(image_path)

# Image inspection function
def inspect_images(image_paths):
    index = 0

    def process_image(button):
        nonlocal index

        quality = button.description
        image_path = image_paths[index]
        img_name = image_path.split(os.sep)[-1]
        if img_name.endswith("gz"):
            img_name.rstrip(".fits.gz")
        else:
            img_name.rstrip(".fits")
        
        if quality == 'Good':
            print(f"Image {img_name} is good!")
            good_images.append(image_path)
        elif quality == 'Strange':
            print(f"Image {img_name} is strange!")
            strange_images.append(image_path)
        elif quality == 'Bad':
            print(f"Image {img_name} is bad!")
            bad_images.append(image_path)

        index += 1
        if index >= len(image_paths):
            print("Inspection completed.")
        else:
            reset_buttons()
            display_image(image_paths[index])

    def reset_buttons():
        for button in quality_buttons:
            button.value = False

    quality_buttons = [
        widgets.Button(description='Good', button_style='success'),
        widgets.Button(description='Strange', button_style='warning'),
        widgets.Button(description='Bad', button_style='danger')
    ]

    buttons_box = widgets.HBox(quality_buttons)
    display(buttons_box)
    display(image_output)

    for button in quality_buttons:
        button.on_click(lambda btn, button=button: process_image(button))

    display_image(image_paths[index])

In [8]:
# List to store information about image quality
good_images = []
strange_images = []
bad_images = []

# Output widget for displaying images
image_output = widgets.Output()

# Start inspection
inspect_images(sorted(coadd_cnt_paths))

Output()

In [9]:
bad_images

[]

In [10]:
glob(f"{root_data_dir}/*/*/*/*-nd-cnt.fits.gz")

['/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29208-KEPLER_SCAN_009_sv08/29208-KEPLER_SCAN_009_sv08_0019-img/KEPLER_SCAN_009_0019_sv08-nd-cnt.fits.gz',
 '/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29208-KEPLER_SCAN_009_sv08/29208-KEPLER_SCAN_009_sv08_0025-img/KEPLER_SCAN_009_0025_sv08-nd-cnt.fits.gz',
 '/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29208-KEPLER_SCAN_009_sv08/29208-KEPLER_SCAN_009_sv08_0024-img/KEPLER_SCAN_009_0024_sv08-nd-cnt.fits.gz',
 '/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29208-KEPLER_SCAN_009_sv08/29208-KEPLER_SCAN_009_sv08_0018-img/KEPLER_SCAN_009_0018_sv08-nd-cnt.fits.gz',
 '/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29208-KEPLER_SCAN_009_sv08/29208-KEPLER_SCAN_009_sv08_0026-img/KEPLER_SCAN_009_0026_sv08-nd-cnt.fits.gz',
 '/Users/julianschliwinski/GALEX_DS/GALEX_DS_GCK_fields/29208-KEPLER_SCAN_009/29